# Predicting a Biological Response

Predict a biological response of molecules from their chemical properties.

## Importing the data

In [20]:
import os
from pathlib import Path
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier

train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

input_X = pd.DataFrame(train.iloc[:, 1:len(train.columns) + 1])
input_y = pd.DataFrame(train.iloc[:, 0])

scaler = None # MinMaxScaler()
pca = None # PCA(random_state=1, n_components=1000)

def get_dataset(X, y, test, scaler = None, pca = None):
    if scaler is not None:
        scaler.fit(X, y)
        X = scaler.transform(X)
        test = scaler.transform(test)
    if pca is not None:
        pca.fit(X, y)
        X = pca.transform(X)
        test = pca.transform(test)
    return X, y, test

X, y, test = get_dataset(input_X, input_y, test, scaler, pca)

In [13]:
classifier = GradientBoostingClassifier(n_estimators=300, random_state=1)
classifier.fit(X, y)

predictions = classifier.predict(X)
predictions_proba = classifier.predict_proba(X)[:, 1]

/home/ronald/anaconda3/envs/minor_data_mining/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
def write_output(predictions):
    indexes = [ n + 1 for n in range(len(predictions) + 1)]
    
    data = list(zip(indexes, predictions))
    columns = ["MoleculeId", "PredictedProbability"]
    results = pd.DataFrame(data, columns=columns)
    
    output_path = Path().absolute().joinpath(Path("predicted.csv"))
    results.to_csv(index=False, path_or_buf=output_path)
    
write_output(predictions_proba)